# Our World in Data COVID-19 data viewer

Antes de começar, é preciso baixar os dados no formato CSV do Our World In Data:

In [1]:
from data import get_data

csv_data = get_data()

Será utilizada a biblioteca pandas para o tratamento dos dados:

In [2]:
import pandas as pd

df = pd.read_csv(csv_data)  # abre os dados em um pandas DataFrame

Escolhe dois dias para calcular a porcentagem da diferença entre os números de casos e depois gerar os gráficos:

In [3]:
first_day = input("Entre a data do primeiro dia no formato YYYY-MM-DD:")

In [4]:
second_day = input("Entre a data do segundo dia no formato YYYY-MM-DD:")

Filtra os dados para cada dia no DataFrame:

In [7]:
first_day_data = df.loc[df["date"] == first_day]
second_day_data = df.loc[df["date"] == second_day]

Informa o número de casos no mundo todo para cada dia, assim como a porcentagem da diferença entre um dia e outro:

In [33]:
first_day_cases = first_day_data.loc[first_day_data["location"] == "World"]["total_cases"].values[0]
second_day_cases = second_day_data.loc[second_day_data["location"] == "World"]["total_cases"].values[0]

print(f"Número de casos em {first_day}: {int(first_day_cases)}")
print(f"Número de casos em {second_day}: {int(second_day_cases)}")

percentage_difference = round((second_day_cases - first_day_cases) / first_day_cases * 100, 2)
print(f"{first_day} --> {second_day}: {'+' if percentage_difference > 0 else ''}{percentage_difference}%")

Número de casos em 2020-04-16: 2043033
Número de casos em 2020-04-17: 2126427
2020-04-16 --> 2020-04-17: +4.08%
